In [1]:
import glob
import os
import json
import nibabel as nib
from skimage.metrics import structural_similarity as ssim
import numpy as np
import tensorflow as tf
from skimage.exposure import rescale_intensity
import pandas as pd
import datetime
import shutil

In [2]:
paths = sorted(glob.glob(r'/content/drive/MyDrive/GP/BraTS Validation Brain Cropped/*'))
Fusion_List = [os.path.basename(path) for path in paths]

In [3]:
root_path = '/content/drive/MyDrive/GP/BraTS Validation Brain Cropped'
data_list = sorted(glob.glob(root_path + '/*'))

fusion_path = '/content/drive/MyDrive/GP/FINAL MODELS/Fusion/50-25-25 T2-T1-T1c'

# Get a list of all the subfolders in the source folder
subfolders = [f.path for f in os.scandir(root_path) if f.is_dir()]

# # # # # Copy each subfolder to the two destination folders
# for folder in subfolders:
#     shutil.copytree(folder, os.path.join(fusion_path, os.path.basename(folder)))

In [ ]:
total_files = 0
for root, dirs, files in os.walk(fusion_path):
    total_files += len(files)

print('Total number of files in folder and subfolders:', total_files)

In [ ]:
#T2_sqz->T1,      T2_Unet->T2
T2_sqz = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1 - 29 July 2023'
T2_unet = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T2 - 29 July 2023'
# T2_sqz = '/content/drive/MyDrive/GP/GANS-FLAIR-55-Mix From T2 - 21 June 2023'
T1c = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1c - 29 July 2023'

sqz_data_list = sorted(glob.glob(T2_sqz + '/*'))
unet_data_list = sorted(glob.glob(T2_unet + '/*'))
t1c_data_list = sorted(glob.glob(T1c + '/*'))
# flair_sqz_data_list = sorted(glob.glob(FLAIR_sqz + '/*'))

T2_SSIM_UNET = []
T2_MAE_UNET = []

T2_SSIM_SQZ = []
T2_MAE_SQZ = []

T2_MAE_FUS = []
T2_SSIM_FUS = []


T2_SQZ_PATHS = []
T2_UNET_PATHS = []
t1c_paths = []
T2_T1_FUSION_PATHS = []
T1_FUSION_PATHS = []
True_paths = []

In [ ]:
for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            True_paths.append(file_path)
True_paths = sorted(True_paths)

for dirpath, dirnames, filenames in os.walk(T2_sqz):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            T2_SQZ_PATHS.append(file_path)
T2_SQZ_PATHS = sorted(T2_SQZ_PATHS)

for dirpath, dirnames, filenames in os.walk(T2_unet):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            T2_UNET_PATHS.append(file_path)
T2_UNET_PATHS = sorted(T2_UNET_PATHS)

for dirpath, dirnames, filenames in os.walk(T1c):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            t1c_paths.append(file_path)
t1c_paths = sorted(t1c_paths)

for dirpath, dirnames, filenames in os.walk(fusion_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            T1_FUSION_PATHS.append(file_path)
            os.remove(file_path)
T1_FUSION_PATHS = sorted(T1_FUSION_PATHS)

In [ ]:
print(T2_SQZ_PATHS[0], (T2_UNET_PATHS[0]), (True_paths[0]), (T1_FUSION_PATHS[0])) #, t1c_paths[0]

/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1 - 29 July 2023/BraTS2021_00000/BraTS2021_00000_flair.nii.gz /content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T2 - 29 July 2023/BraTS2021_00000/BraTS2021_00000_flair.nii.gz /content/drive/MyDrive/GP/BraTS Validation Brain Cropped/BraTS2021_00000/BraTS2021_00000_flair.nii.gz /content/drive/MyDrive/GP/FINAL MODELS/Fusion/50-25-25 T2-T1-T1c/BraTS2021_00000/BraTS2021_00000_flair.nii.gz


In [ ]:
print(len(T2_SQZ_PATHS), len(T2_UNET_PATHS), len(True_paths), len(T1_FUSION_PATHS)) #, len(t1c_paths)

251 251 251 251


In [ ]:
# calculate the SSIM score for each slice of each pair of volumes
def calc_2D_tf_SSIM(Synth_paths, True_paths):
    SSIM_tf_2D = []
    for path1, path2 in zip(Synth_paths, True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()
        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))

        # calculate the SSIM score for each slice
        ssim_scores_vol = []

        for i in range(vol1.shape[2]):
            slice1 = vol1[:, :, i]
            slice2 = vol2[:, :, i]

            slice1 = np.stack([slice1]*3, axis=-1)
            slice2 = np.stack([slice2]*3, axis=-1)

            ssim_score = tf.image.ssim(slice1, slice2, max_val=1)
            ssim_scores_vol.append(ssim_score)


        # calculate the average SSIM score for the volume
        avg_ssim_vol = np.mean(ssim_scores_vol)

        # add the average SSIM scores to the list
        SSIM_tf_2D.append(avg_ssim_vol)

    return SSIM_tf_2D

def _calculate_mae_mse(arr1, arr2):
    diff = arr1 - arr2
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff ** 2)
    return mae, mse

def calculate_mae_mse(T1_Synth_paths, T1_True_paths, norm_range):
    MAE, MSE = [], []
    for path1, path2 in zip(T1_Synth_paths, T1_True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()

        vol1 = rescale_intensity(vol1, out_range=norm_range)
        vol2 = rescale_intensity(vol2, out_range=norm_range)

        mae, mse = _calculate_mae_mse(vol1, vol2)

        MAE.append(mae)
        MSE.append(mse)
    return MAE, MSE

def psnr(Synth_paths, True_paths):
    """
    Calculate the average PSNR between corresponding slices of two 3D volumes.
    """
    PSNR = []
    for path1, path2 in zip(Synth_paths, True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()
        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))

        psnrs = []
        for i in range(vol1.shape[2]):
            mse = np.mean((vol1[:,:,i] - vol2[:,:,i]) ** 2)
            if mse == 0:
                psnrs.append(float('inf'))
            else:
                max_pixel = np.max(vol1[:,:,i])
                psnrs.append(20 * np.log10(max_pixel / np.sqrt(mse)))
        avg_psnr = np.mean(psnrs)
        PSNR.append(avg_psnr)
    return PSNR

In [ ]:
def three_two_fusion(vol1_paths, vol2_paths, vol3_paths):
    i = 0
    for path1, path2, path3 in zip(vol1_paths, vol2_paths, vol3_paths):
        # load the volumes from files
        x = nib.load(True_paths[i])
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()
        vol3 = nib.load(path3).get_fdata()

        # Convert the data arrays to a suitable data type for averaging
        vol1 = vol1.astype(np.float32)
        vol2 = vol2.astype(np.float32)
        vol3 = vol3.astype(np.float32)

        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))
        vol3 = rescale_intensity(vol3, out_range=(0, 1))

        vol4 = (vol1 + vol2 + vol3 + vol3) / 4.0
        # vol4 = (vol1 + vol2 + vol3) / 3.0
        # create a new NIfTI image object with the processed data
        nifti_image_processed = nib.Nifti1Image(vol4, x.affine, x.header)
        # save the processed NIfTI file
        nib.save(nifti_image_processed, T1_FUSION_PATHS[i])
        i = i + 1
    return vol4, nifti_image_processed

x, y = three_two_fusion(T2_SQZ_PATHS, T2_UNET_PATHS, t1c_paths)

In [ ]:
def three_fusion(vol1_paths, vol2_paths, vol3_paths):
    i = 0
    for path1, path2, path3 in zip(vol1_paths, vol2_paths, vol3_paths):
        # load the volumes from files
        x = nib.load(True_paths[i])
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()
        vol3 = nib.load(path3).get_fdata()

        # Convert the data arrays to a suitable data type for averaging
        vol1 = vol1.astype(np.float32)
        vol2 = vol2.astype(np.float32)
        vol3 = vol3.astype(np.float32)

        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))
        vol3 = rescale_intensity(vol3, out_range=(0, 1))

        vol4 = (vol1 + vol2 + vol3) / 3.0
        # create a new NIfTI image object with the processed data
        nifti_image_processed = nib.Nifti1Image(vol4, x.affine, x.header)
        # save the processed NIfTI file
        nib.save(nifti_image_processed, T1_FUSION_PATHS[i])
        i = i + 1
    return vol4, nifti_image_processed

x, y = three_fusion(T2_SQZ_PATHS, T2_UNET_PATHS, t1c_paths)

In [ ]:
def fusion(vol1_paths, vol2_paths):
    i = 0
    for path1, path2 in zip(vol1_paths, vol2_paths):
        # load the volumes from files
        x = nib.load(True_paths[i])
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()

        # Convert the data arrays to a suitable data type for averaging
        vol1 = vol1.astype(np.float32)
        vol2 = vol2.astype(np.float32)

        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))

        vol3 = (vol1 + vol2) / 2.0
        # create a new NIfTI image object with the processed data
        nifti_image_processed = nib.Nifti1Image(vol3, x.affine, x.header)
        # save the processed NIfTI file
        nib.save(nifti_image_processed, T1_FUSION_PATHS[i])
        i = i + 1
    return vol3, nifti_image_processed

x, y = fusion(T2_SQZ_PATHS, T2_UNET_PATHS)

In [ ]:
total_files = 0
for root, dirs, files in os.walk(fusion_path):
    total_files += len(files)

print('Total number of files in folder and subfolders:', total_files)

Total number of files in folder and subfolders: 1255


In [ ]:
# T2_SSIM_SQZ = calc_2D_tf_SSIM(T2_SQZ_PATHS, True_paths)
# T2_MAE_SQZ, _ = calculate_mae_mse(T2_SQZ_PATHS, True_paths, norm_range=(0, 1))

# T2_SSIM_UNET = calc_2D_tf_SSIM(T2_UNET_PATHS, True_paths)
# T2_MAE_UNET, _ = calculate_mae_mse(T2_UNET_PATHS, True_paths, norm_range=(0, 1))

T2_SSIM_FUS = calc_2D_tf_SSIM(T1_FUSION_PATHS, True_paths)
T2_MAE_FUS, _ = calculate_mae_mse(T1_FUSION_PATHS, True_paths, norm_range=(0, 1))

In [ ]:
T2_SSIM_SQZ = calc_2D_tf_SSIM(T2_SQZ_PATHS, True_paths)
T2_MAE_SQZ, _ = calculate_mae_mse(T2_SQZ_PATHS, True_paths, norm_range=(0, 1))

T2_SSIM_UNET = calc_2D_tf_SSIM(T2_UNET_PATHS, True_paths)
T2_MAE_UNET, _ = calculate_mae_mse(T2_UNET_PATHS, True_paths, norm_range=(0, 1))

In [ ]:
print(len(Fusion_List), len(T2_SSIM_SQZ), len(T2_SSIM_UNET), len(T2_SSIM_FUS), len(T2_MAE_SQZ), len(T2_MAE_UNET), len(T2_MAE_FUS))

251 0 0 251 0 0 251


In [ ]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    "Validation Set": Fusion_List,
    'SSIM FLAIR from T1': T2_SSIM_SQZ,
    'SSIM FLAIR from T1c': T2_SSIM_UNET,
    'SSIM FLAIR Fusion': T2_SSIM_FUS,

    'MAE FLAIR from T1': T2_MAE_SQZ,
    'MAE FLAIR from T1c': T2_MAE_UNET,
    'MAE FLAIR Fusion': T2_MAE_FUS
})

csv_path = '/content/drive/MyDrive/GP/FLAIR GANs Fusion/FLAIR-55ep - 50-25-25 T2+T1+T1c.csv'
# Save the DataFrame to a CSV file
df.to_csv(csv_path, index=False)